# SmartVote-Lite
## Blockchain-Based Voting System with AI Anomaly Detection
## Final Year Project – CSE (Anna University)



### Objective
To design and implement a secure blockchain-based voting system and
integrate an AI-based anomaly detection model to identify suspicious
voting behavior such as vote duplication and sudden vote spikes.


### Technologies Used
- Python 3
- Jupyter Notebook
- Blockchain (SHA-256 hashing)
- Machine Learning (Isolation Forest)
- Pandas (Data Processing)


In [1]:

### Step 1: Import Required Libraries
import hashlib
import time
import random
import pandas as pd
from sklearn.ensemble import IsolationForest


In [2]:
 ### step 2: Define Block Class
class Block:
    def __init__(self, index, data, previous_hash):
        self.index = index
        self.timestamp = time.time()
        self.data = data
        self.previous_hash = previous_hash
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        content = f"{self.index}{self.timestamp}{self.data}{self.previous_hash}"
        return hashlib.sha256(content.encode()).hexdigest()


In [3]:
### step 3: Defince Blockchain Class
class Blockchain:
    def __init__(self):
        self.chain = [self.create_genesis_block()]

    def create_genesis_block(self):
        return Block(0, "Genesis Block", "0")

    def get_latest_block(self):
        return self.chain[-1]

    def add_block(self, data):
        block = Block(
            index=len(self.chain),
            data=data,
            previous_hash=self.get_latest_block().hash
        )
        self.chain.append(block)

    def display_chain(self):
        for block in self.chain:
            print("Index:", block.index)
            print("Hash:", block.hash)
            print("Data:", block.data)
            print("-" * 40)



In [4]:
### Initialize Blockchain & Vote Log
blockchain = Blockchain()
vote_log = []


In [5]:
### step 5: Simulate Real-Time Voting (WITH SPIKE)

regions = ["Chennai", "Madurai", "Coimbatore"]
candidates = ["A", "B"]

for i in range(120):   # simulate large-scale voting
    vote = {
        "voter_id": f"TN{i+1000}",
        "region": random.choice(regions),
        "candidate": random.choice(candidates),
        "timestamp": time.time()
    }

    blockchain.add_block(vote)
    vote_log.append(vote)

    # Artificial abnormal spike in Chennai
    if i > 70 and vote["region"] == "Chennai":
        time.sleep(0.05)   # abnormal fast votes
    else:
        time.sleep(0.3)    # normal voting speed


In [7]:
### step6: convert vote log to dataframe
df = pd.DataFrame(vote_log)
df["time_diff"] = df["timestamp"].diff().fillna(0)
df.head()


,voter_id,region,candidate,timestamp,time_diff
0,TN1000,Coimbatore,B,1.767426e+09,0.000000
1,TN1001,Chennai,A,1.767426e+09,0.305132
2,TN1002,Coimbatore,A,1.767426e+09,0.305225
3,TN1003,Madurai,A,1.767426e+09,0.302854
4,TN1004,Coimbatore,B,1.767426e+09,0.301918


In [8]:
### step7:AI Anomaly Detection
X = df[["time_diff"]]

model = IsolationForest(contamination=0.15, random_state=42)
df["anomaly"] = model.fit_predict(X)

df["status"] = df["anomaly"].apply(
    lambda x: "Suspicious Spike" if x == -1 else "Normal"
)



In [11]:
### step8: Display Suspicious Votes
df[df["status"] == "Suspicious Spike"]


,voter_id,region,candidate,timestamp,time_diff,anomaly,status
0,TN1000,Coimbatore,B,1.767426e+09,0.000000,-1,Suspicious Spike
10,TN1010,Coimbatore,B,1.767426e+09,0.300649,-1,Suspicious Spike
11,TN1011,Chennai,A,1.767426e+09,0.300401,-1,Suspicious Spike
17,TN1017,Madurai,B,1.767426e+09,0.300902,-1,Suspicious Spike
48,TN1048,Madurai,B,1.767426e+09,0.308795,-1,Suspicious Spike
49,TN1049,Coimbatore,B,1.767426e+09,0.304208,-1,Suspicious Spike
61,TN1061,Chennai,A,1.767426e+09,0.300790,-1,Suspicious Spike
78,TN1078,Coimbatore,A,1.767426e+09,0.055174,-1,Suspicious Spike
84,TN1084,Coimbatore,A,1.767426e+09,0.053587,-1,Suspicious Spike
88,TN1088,Coimbatore,A,1.767426e+09,0.055170,-1,Suspicious Spike


In [13]:
###step9: Display Blockchain
blockchain.display_chain()


Index: 0
Hash: af028aee1df12376e953ca4fb91009ac26ee302ba638694d2a713ce2571c3128
Data: Genesis Block
----------------------------------------
Index: 1
Hash: 6710cb2e8c5d3a512c739c3d0cd1b692d3107efac93fde3994337ffd51cad88c
Data: {'voter_id': 'TN1000', 'region': 'Coimbatore', 'candidate': 'B', 'timestamp': 1767425963.794434}
----------------------------------------
Index: 2
Hash: 0a737320fcedbc6cf084caa921638c5e3f5d7c6b811deb602bc88e4d7afc94ad
Data: {'voter_id': 'TN1001', 'region': 'Chennai', 'candidate': 'A', 'timestamp': 1767425964.099566}
----------------------------------------
Index: 3
Hash: 71e3f641f501bce114498e7396b3c1347dc273f83e7e9b0d84fe96d01f0486b8
Data: {'voter_id': 'TN1002', 'region': 'Coimbatore', 'candidate': 'A', 'timestamp': 1767425964.404791}
----------------------------------------
Index: 4
Hash: 1c0171d9809c6445145f7f997e24eb077c7a1d6e0c37aefda58a125add8a7d74
Data: {'voter_id': 'TN1003', 'region': 'Madurai', 'candidate': 'A', 'timestamp': 1767425964.707645}
----------